# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv
import time

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core import Environment, ScriptRunConfig
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.pipeline.core import Pipeline


# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.51.0


In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'hyperdrive-experiment'

experiment= Experiment(ws, experiment_name)

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "my-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
# For a more detailed view of current AmlCompute status, use get_status().

## Dataset


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [6]:
import sys

project_name = 'azureml-spaceship-titanic'
project_folder = os.getcwd().split(project_name)[0] + project_name
sys.path.append(project_folder)

from src.pipelines.preprocess import preprocess_data


df = pd.read_csv('../data/01_raw/train.csv')
processed_df = preprocess_data(df).drop(columns=["PassengerId"])


In [7]:
dataset_name = "Spaceship_Dataset"
description_text = "Data to predict which passengers are transported to an alternate dimension"

# Create AML Dataset and register it into Workspace
dataset = Dataset.Tabular.register_pandas_dataframe(processed_df, ws.get_default_datastore(), dataset_name)   
#Register Dataset in Workspace
dataset = dataset.register(workspace=ws,
                                name=dataset_name,
                                description=description_text,
                                create_new_version=True
                        )


df = dataset.to_pandas_dataframe()
df.describe()

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/8183cbb2-ea0b-4fa1-89f8-c9a06d6ea7a2/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,People_in_Cabin_Num,People_in_Cabin_Deck,Family_Size,Group_Size
count,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000,8494.000000,8494.000000,8493.000000,8693.000000
mean,224.687617,458.077203,173.729169,311.138778,304.854791,8.037203,1952.105957,5.428117,2.035546
std,666.717663,1611.489240,604.696458,1136.705535,1145.717189,5.214669,963.741893,2.891901,1.596347
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,5.000000,1.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,779.000000,3.000000,1.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,2559.000000,5.000000,1.000000
75%,47.000000,76.000000,27.000000,59.000000,46.000000,12.000000,2794.000000,7.000000,3.000000
max,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000,28.000000,2794.000000,18.000000,8.000000


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform

In [30]:
dataset_name = 'Spaceship_Dataset'
aml_dataset = Dataset.get_by_name(workspace=ws, name=dataset_name)

KeyboardInterrupt: 

In [29]:
aml_dataset

Workspace.create(name='quick-starts-ws-250931', subscription_id='f9d5a085-54dc-4215-9ba6-dad5d86e60a0', resource_group='aml-quickstarts-250931')

In [26]:
import os
from azureml.core import Environment, ScriptRunConfig, Experiment
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal, choice
from azureml.widgets import RunDetails

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
# I noticed you didn't define 'policy', so I'm using the 'early_termination_policy' you defined earlier.
early_termination_policy = BanditPolicy(slack_factor=0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

sklearn_env = Environment.get(workspace=ws, name="my_env")

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory="./",
    script="train.py",  # Replace with the name of your training script
    compute_target=compute_target,  # Replace with your compute target
    environment=sklearn_env
)

# TODO: Create the different params that you will be using during training
ps = RandomParameterSampling({
    "--C": choice(0.001, 0.01, 0.1, 1, 10, 100, 1000),
    "--max_iter": choice(100, 150, 200, 250, 500, 1000)
})

# TODO: Create your estimator and hyperdrive config
hyperdrive_run_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    policy=early_termination_policy,
    primary_metric_name="Accuracy",  # Match this with the metric name logged in train.py
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=4
)

# Submit the HyperDrive run
hyperdrive_run = experiment.submit(hyperdrive_run_config)

# Monitor the HyperDrive run
RunDetails(hyperdrive_run).show()

# Wait for HyperDrive run completion
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_2738e829-3421-4d91-88f1-7c146df1130d
Web View: https://ml.azure.com/runs/HD_2738e829-3421-4d91-88f1-7c146df1130d?wsid=/subscriptions/f9d5a085-54dc-4215-9ba6-dad5d86e60a0/resourcegroups/aml-quickstarts-250931/workspaces/quick-starts-ws-250931&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2024-01-29T04:01:51.783824][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2024-01-29T04:01:52.3323480Z][SCHEDULER][INFO]Scheduling job, id='HD_2738e829-3421-4d91-88f1-7c146df1130d_0' 
[2024-01-29T04:01:52.4717777Z][SCHEDULER][INFO]Scheduling job, id='HD_2738e829-3421-4d91-88f1-7c146df1130d_1' 
[2024-01-29T04:01:52.5847014Z][SCHEDULER][INFO]Scheduling job, id='HD_2738e829-3421-4d91-88f1-7c146df1130d_2' 
[2024-01-29T04:01:52.6085754Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_2738e829-3421-4d91-88f1-7c146df1130d_0' 
[2024-01-29T04:01:52.7669713Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_2738e829-3421-4d91-88f1-

ExperimentExecutionException: ExperimentExecutionException:
	Message: The output streaming for the run interrupted.
But the run is still executing on the compute target. 
Details for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "The output streaming for the run interrupted.\nBut the run is still executing on the compute target. \nDetails for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run"
    }
}

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

